# imports


In [8]:
import os
import gradio as gr
import openai
from gtts import gTTS
import tempfile
from dotenv import load_dotenv
import speech_recognition as sr
import io

In [9]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [10]:
client = openai.OpenAI()
MODEL = "gpt-4o-mini"

In [11]:
SYSTEM_MESSAGE = """
You are an advanced EV Battery & Vehicle Assistant specialized in answering queries about electric vehicles (EVs) in India, including brands like Tata, Mahindra, MG, Hyundai, Kia, BYD, and others.
Your purpose is to provide accurate, clear, and useful information based only on official EV manuals, technical documentation, verified specifications, and authentic sources.

Guidelines:

Cover all aspects of EVs in India: battery (SoH, RUL, charging cycles, warnings, troubleshooting), vehicle performance, charging infrastructure, warranty, and usage best practices.

When answering, always use simple, human-like language and avoid unnecessary jargon.

If a query is vague, interpret it intelligently and provide the most relevant and helpful explanation instead of rejecting it.

Stay factual and grounded in available manuals, datasets, or trusted references—never invent details.

Structure answers in a way that is easy to follow, with step-by-step explanations when needed.

Response Format:
Answer: [Clear and concise explanation]
Reference: [Mention the manual/section/source if available]
"""


In [12]:
def get_answer(user_text):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_MESSAGE},
            {"role": "user", "content": user_text}
        ]
    )
    answer = response.choices[0].message.content

    # Convert text to speech
    tts = gTTS(text=answer, lang="en")
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
        tts.save(fp.name)
        audio_path = fp.name

    return answer, audio_path

In [ ]:
def chatbot_audio(audio_file):
    print(f"Audio file received: {audio_file}")  # Debug print
    
    if audio_file is None:
        return "⚠️ No audio file received. Please record audio first!", None
    
    try:
        print(f"Processing audio file: {audio_file}")
        
        # Check if file exists and has content
        if not os.path.exists(audio_file):
            return "❌ Audio file not found!", None
            
        file_size = os.path.getsize(audio_file)
        print(f"Audio file size: {file_size} bytes")
        
        if file_size == 0:
            return "❌ Audio file is empty. Please record again!", None
        
        # Step 1: Transcribe audio to text using Whisper
        with open(audio_file, "rb") as f:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=f,
                response_format="text"
            )
        
        user_text = transcript.strip() if isinstance(transcript, str) else transcript.text.strip()
        print(f"Transcribed text: {user_text}")
        
        if not user_text:
            return "❌ Could not understand the audio. Please speak clearly and try again!", None
        
        # Step 2: Get answer from chatbot
        return get_answer(user_text)
        
    except Exception as e:
        error_msg = f"❌ Audio processing error: {str(e)}"
        print(error_msg)
        return error_msg, None

In [14]:
def chatbot_text(user_input):
    return get_answer(user_input)

In [ ]:
# Simple Voice Interface - WORKING VERSION
with gr.Blocks(title="EV Voice Assistant") as demo:
    gr.Markdown("## 🔋 EV Battery Twin Assistant (Voice Enabled)")
    gr.Markdown("### 🎤 Record your question and click Submit!")
    
    with gr.Row():
        with gr.Column(scale=1):
            mic_input = gr.Audio(
                sources=["microphone"], 
                type="filepath", 
                label="🎤 Record Your Question",
                show_download_button=False
            )
            
            speech_btn = gr.Button("🎤 Process Voice", variant="primary", size="lg")
            
        with gr.Column(scale=1):
            text_input = gr.Textbox(
                label="💬 Or Type Your Question", 
                placeholder="Ask about Tata EVs...",
                lines=3
            )
            
            text_btn = gr.Button("📝 Send Text", variant="secondary", size="lg")

    with gr.Row():
        text_output = gr.Textbox(label="🤖 Assistant Answer", lines=5)
        
    with gr.Row():
        audio_output = gr.Audio(label="🔊 Listen to Response", autoplay=True)

    # Handle speech input
    speech_btn.click(
        fn=chatbot_audio, 
        inputs=mic_input, 
        outputs=[text_output, audio_output],
        show_progress=True
    )

    # Handle text input  
    text_btn.click(
        fn=chatbot_text, 
        inputs=text_input, 
        outputs=[text_output, audio_output],
        show_progress=True
    )
    
    # Also allow Enter key for text
    text_input.submit(
        fn=chatbot_text, 
        inputs=text_input, 
        outputs=[text_output, audio_output]
    )

    gr.Markdown("---")
    gr.Markdown("**🎯 How to use:** Record audio → Click '🎤 Process Voice' → Get answer!")

demo.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "d:\kpr-hackathon\chat\lib\site-packages\gradio\queueing.py", line 745, in process_events
    response = await route_utils.call_process_api(
  File "d:\kpr-hackathon\chat\lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
  File "d:\kpr-hackathon\chat\lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
  File "d:\kpr-hackathon\chat\lib\site-packages\gradio\blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "d:\kpr-hackathon\chat\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "d:\kpr-hackathon\chat\lib\site-packages\anyio\_backends\_asyncio.py", line 2485, in run_sync_in_worker_thread
    return await future
  File "d:\kpr-hackathon\chat\lib\site-packages\anyio\_backends\_asyncio.py", 

In [ ]:
# QUICK AUDIO TEST - Run this first!
def test_audio_recording(audio_file):
    if audio_file is None:
        return "❌ No audio recorded"
    
    if os.path.exists(audio_file):
        size = os.path.getsize(audio_file)
        return f"✅ Audio recorded successfully! File size: {size} bytes"
    else:
        return "❌ Audio file not found"

with gr.Blocks() as audio_test:
    gr.Markdown("## 🧪 Audio Recording Test")
    gr.Markdown("Test if your microphone is working:")
    
    test_audio = gr.Audio(sources=["microphone"], type="filepath", label="Record Test Audio")
    test_btn = gr.Button("Test Recording")
    test_result = gr.Textbox(label="Test Result")
    
    test_btn.click(test_audio_recording, inputs=test_audio, outputs=test_result)

# Uncomment to run the test:
# audio_test.launch()

# Just ChatBot

In [34]:
def chatbot_text(user_input, history):
    messages = [{"role": "system", "content": SYSTEM_MESSAGE}]
    for user, bot in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": bot})

    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    answer = response.choices[0].message.content

    history.append((user_input, answer))
    return history, history, ""

In [38]:
with gr.Blocks() as demo:
    gr.Markdown("## Flexi EV Assistant")

    chatbot = gr.Chatbot(label="Chat with Flexi")
    text_input = gr.Textbox(label="Ask your question", placeholder="Type here...")

    state = gr.State([])

    text_input.submit(
        chatbot_text,
        
        inputs=[text_input, state],
        outputs=[chatbot, state, text_input]  # <--- include text_input here
    )

demo.launch()

C:\Users\sajit\AppData\Local\Temp\ipykernel_42108\3500426637.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat with Flexi")


* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


In [18]:
# Complete Speech-Enabled Chatbot with History
def chatbot_with_speech(user_input, audio_input, history):
    # Handle audio input if provided
    if audio_input is not None:
        try:
            with open(audio_input, "rb") as f:
                transcript = client.audio.transcriptions.create(
                    model="whisper-1",
                    file=f
                )
            user_input = transcript.text
        except Exception as e:
            return history + [("Audio Error", f"Could not process audio: {str(e)}")], history, "", None
    
    if not user_input or not user_input.strip():
        return history, history, "", None
    
    # Build conversation context
    messages = [{"role": "system", "content": SYSTEM_MESSAGE}]
    for user, bot in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": bot})
    
    messages.append({"role": "user", "content": user_input})
    
    try:
        # Get AI response
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        answer = response.choices[0].message.content
        
        # Convert answer to speech
        tts = gTTS(text=answer, lang="en")
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
            tts.save(fp.name)
            audio_path = fp.name
        
        # Update history
        new_history = history + [(user_input, answer)]
        return new_history, new_history, "", audio_path
        
    except Exception as e:
        error_msg = f"Error: {str(e)}"
        new_history = history + [(user_input, error_msg)]
        return new_history, new_history, "", None

# Launch the complete interface
with gr.Blocks(theme=gr.themes.Soft()) as speech_demo:
    gr.Markdown("# SmartEV Assistant - Voice & Text Enabled")
    gr.Markdown("Ask questions about EVs using **voice** or **text**!")
    gr.Markdown(" **Allow microphone access** when prompted by your browser!")
    
    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                label="EV Assistant Chat", 
                height=400,
                show_copy_button=True
            )
        
        with gr.Column(scale=1):
            audio_output = gr.Audio(
                label=" Listen to Response",
                autoplay=False,
                show_download_button=True
            )
    
    with gr.Row():
        with gr.Column(scale=2):
            text_input = gr.Textbox(
                label=" Type your question",
                placeholder="Ask about EV batteries, charging, maintenance...",
                lines=2
            )
        
        with gr.Column(scale=2):
            audio_input = gr.Audio(
                sources=["microphone"],
                type="filepath",
                label="🎤 Record Question",
                format="wav",
                show_download_button=False
            )
    
    with gr.Row():
        submit_btn = gr.Button("Send Message", variant="primary", size="lg")
        voice_btn = gr.Button("🎤 Send Voice", variant="secondary", size="lg")
        clear_btn = gr.Button("Clear Chat", variant="stop")
    
    # State to maintain conversation history
    chat_state = gr.State([])
    
    # Handle text input
    submit_btn.click(
        chatbot_with_speech,
        inputs=[text_input, gr.State(None), chat_state],
        outputs=[chatbot, chat_state, text_input, audio_output]
    )
    
    # Handle voice input
    voice_btn.click(
        chatbot_with_speech,
        inputs=[gr.State(""), audio_input, chat_state],
        outputs=[chatbot, chat_state, text_input, audio_output]
    )
    
    # Also allow Enter key for text input
    text_input.submit(
        chatbot_with_speech,
        inputs=[text_input, gr.State(None), chat_state],
        outputs=[chatbot, chat_state, text_input, audio_output]
    )
    
    # Clear chat history
    clear_btn.click(
        lambda: ([], []),
        outputs=[chatbot, chat_state]
    )
    
    gr.Markdown("---")
    gr.Markdown("** Usage Tips:**")
    gr.Markdown("- **Text:** Type and press Enter or click 'Send Message'")
    gr.Markdown("- **Voice:** Record audio, then click '🎤 Send Voice'")
    gr.Markdown("- **Browser:** Make sure to allow microphone permissions!")

speech_demo.launch(share=True, debug=True)

C:\Users\sajit\AppData\Local\Temp\ipykernel_10348\2396453029.py:57: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://bfbd8fb3f3b5f22da9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
* Running on public URL: https://bfbd8fb3f3b5f22da9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


: 

In [16]:
# Quick Microphone Test
def test_microphone():
    import gradio as gr
    
    def process_test_audio(audio):
        if audio is None:
            return "❌ No audio received. Check microphone permissions!"
        else:
            return f"✅ Audio received! File: {audio}"
    
    with gr.Blocks() as mic_test:
        gr.Markdown("## 🎤 Microphone Test")
        gr.Markdown("Record a short audio to test if microphone works:")
        
        test_audio = gr.Audio(
            sources=["microphone"], 
            type="filepath", 
            label="Test Recording",
            format="wav"
        )
        test_btn = gr.Button("Test Audio")
        result = gr.Textbox(label="Test Result")
        
        test_btn.click(process_test_audio, inputs=test_audio, outputs=result)
    
    return mic_test

# Uncomment to run microphone test
# mic_test_demo = test_microphone()
# mic_test_demo.launch()

In [17]:
audio_test.launch()

NameError: name 'audio_test' is not defined